In [ ]:
cfg_name = 'cfg20200325T2048'
model_epoch='0025'
print("L")

In [ ]:
cfg_name = 'cfg20200325T2048'
model_epoch='0025'
print("cfg_name:",cfg_name)
import pickle
import os
import cv2
with open('rcnn_100_random_images_ferebee.pkl', 'rb') as f:
    test_file_names = pickle.load(f)


image_prediction_folder = '/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/predictions/'+cfg_name+"/unknown/"
image_for_prediction_folder = '/storage/box/Pompeii in Pictures/Pompeii Ferebee/Pompeii/'    
# display image with masks and bounding boxes
from os import listdir
import skimage
import random
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

# set up font for boxes
# https://www.geeksforgeeks.org/python-opencv-cv2-puttext-method/
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
font_scale = 3
font_color = (255, 255, 255) 
font_thickness = 8
   

import cv2
for cn in range(0,100):
    
    print("jeff",cn)
    image = skimage.io.imread(os.path.join(image_for_prediction_folder, test_file_names[cn]))
    
    obj_f_num = "00000"+str(cn)
    obj_f_num = obj_f_num[-5:]
    #print(fn)
    tree = ElementTree.parse(image_prediction_folder+obj_f_num+'.xml')
    print(image_prediction_folder+obj_f_num+'.xml')
    print(tree)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    
    fn_image = root.find('./filename').text
    object_present = root.find('./object_present').text
    #print(fn_image)
    box_num=0
    overlay = image.copy()
    output = image.copy()
    boxes_correct={}
    for obj in root.findall('./object'):
        score = obj.find('score').text
        correct = obj.find('correct').text
        if(correct is None):
            correct="0"
        if(correct==""):
            correct="0"
        correct=int(correct)
        box = obj.find('bndbox')
        box_num=obj.find('number').text
        boxes_correct[str(box_num)] = correct
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        if(ymin>ymax):
            ytemp = ymin
            ymin = ymax
            ymax=ytemp
        if(xmin>xmax):
            xtemp = xmin
            xmin = xmax
            xmax=xtemp            
        coors = [xmin, ymin, xmax, ymax]
        print(coors)
        if(correct==0):
            cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(0, 0, 255), 10)
        elif(correct==1):
            cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(0, 255,0 ), 10)
        # Using cv2.putText() method 
        cv2.putText(output, '['+box_num+'] S: '+ score[:5], (xmin+10, ymin+70), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 

    alpha = .4
    #cv2.addWeighted(overlay, alpha, output, 1-alpha,0, output)
    cv2.putText(output, '['+str(cn)+'] Present: '+ object_present, (50, 70), font, font_scale, font_color, font_thickness, cv2.LINE_AA) 
    image = output
    scale_percent = 1024/image.shape[1] # percent of original size
    #print(scale_percent)
    width = int(image.shape[1] * scale_percent )
    height = int(image.shape[0] * scale_percent )
    dim = (width, height)
    # resize image
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    resized_original = resized.copy()
    
    object_file_num_int_return = -1
 # keep looping until the 'c' key is pressed
    while True:
        # display the image and wait for a keypress
        cv2.imshow("image", resized)
        cv2.moveWindow("image", 0,100)
        key = cv2.waitKey(1) & 0xFF

        # if the 'c' key is pressed, break from the loop
        if key == ord("n"):
            object_present = "0"
        if key == ord("p"):
            object_present = "1"
            
            
        if key == ord("w"):
            object_file_num_int_return = -1
            break
        if key == ord("+"):
            break
        if key == ord("-"):
            cn = cn -1
            break
        if key == ord("c"):
            break
            
        if key == ord("0"):
            boxes_correct["0"]="1"
        if key == ord("1"):
            boxes_correct["1"]="1"
        if key == ord("2"):
            boxes_correct["2"]="1"
        if key == ord("3"):
            boxes_correct["3"]="1"
        if key == ord("4"):
            boxes_correct["4"]="1"
        if key == ord("5"):
            boxes_correct["5"]="1"
        if key == ord("6"):
            boxes_correct["6"]="1"
        if key == ord("7"):
            boxes_correct["7"]="1"
        if key == ord("8"):
            boxes_correct["8"]="1"
        if key == ord("9"):
            boxes_correct["9"]="1"
        if key == ord("A"):
            boxes_correct["10"]="1"

    if object_file_num_int_return == -1:
        #correct_number = root.find('./correct_number')
        for obj in root.findall('./object'):
            box_num=obj.find('number').text
            correct = obj.find('correct')
            correct.text = str(boxes_correct[str(box_num)])
            print(boxes_correct)
            print(boxes_correct[str(box_num)])
        #correct_number.text = str(object_file_num_int_return)
        #root.set('correct_number',str(object_file_num_int_return))
        
        op = root.find('object_present')  #.text = "xx" #object_present
        op.text = str(object_present)
        tree.write(image_prediction_folder+obj_f_num+'.xml')
            
            
    # close all open windows
    cv2.destroyAllWindows()
    image=None
    if key == ord("c"):
        break

# Gather the data from the xml files into results_cfg_.csv

In [ ]:
print("cfg_name:",cfg_name)
image_prediction_folder = '/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/predictions/'+cfg_name+"/unknown/"

# display image with masks and bounding boxes
from os import listdir
from xml.etree import ElementTree

results = open(image_prediction_folder+"results_"+cfg_name+".csv","w+")
results.write("min_score,gt_obj_pres_pos,gt_obj_pres_neg,tp,fp,tn,fn,tp_box,fp_box\n")
min_scores = []
min_scores.append(.7)
min_scores.append(.75)
min_scores.append(.80)
min_scores.append(.85)
min_scores.append(.90)
min_scores.append(.95)
for min_score in min_scores:
    gt_obj_pres_pos = 0
    gt_obj_pres_neg = 0
    tp=0
    fp=0
    tn=0
    fn=0
    tp_box=0
    fp_box=0

    

    for cn in range(0,100):
    
        obj_f_num = "00000"+str(cn)
        obj_f_num = obj_f_num[-5:]
        #print(fn)
        tree = ElementTree.parse(image_prediction_folder+obj_f_num+'.xml')
        print(image_prediction_folder+obj_f_num+'.xml')
        print(tree)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
    

    
        #print(fn_image)
        box_num=0
        boxes_correct={}
        obj_pres_pred = 0
        for obj in root.findall('./object'):
            score = obj.find('score').text
            score = float(score)
            if score>=min_score:
                correct = obj.find('correct').text
                if(correct is None):
                    correct="0"
                if(correct==""):
                    correct="0"
                correct=int(correct)
                if(correct==0):
                    fp_box=fp_box+1
                if(correct==1):
                    tp_box=tp_box+1
                #if one is correct, it's deemed found in the picture.
                obj_pres_pred = 1
                #obj_pres_pred or correct
                print(correct,obj_pres_pred)
        object_present = root.find('./object_present').text
        if(object_present=="1"):
            gt_obj_pres_pos = gt_obj_pres_pos + 1
            if(obj_pres_pred==1):
                tp=tp+1
            if(obj_pres_pred==0):
                fn=fn+1
        if(object_present=="0"):
            gt_obj_pres_neg = gt_obj_pres_neg + 1
            if(obj_pres_pred==1):
                fp=fp+1
            if(obj_pres_pred==0):
                tn=tn+1
    print("gt_obj_pres_pos", gt_obj_pres_pos)
    print("gt_obj_pres_neg", gt_obj_pres_neg)
    #results.write("cfg_name,gt_obj_pres,gt_obj_not_pres,TP,FP,TN,FN\n")
    results.write(str(min_score)+","+str(gt_obj_pres_pos)+","+str(gt_obj_pres_neg)+","+str(tp)+","+str(fp)+","+str(tn)+","+str(fn)+","+str(tp_box)+","+str(fp_box)+"\n")

results.close()